In [5]:
import wandb

wandb.init(project="example", entity="xdss")

In [6]:
from sklearn.datasets import make_classification
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelBinarizer

X, y = make_classification(n_samples=10000, n_informative=10, n_classes=3)

lb = LabelBinarizer().fit(y)

In [11]:
import optuna
import lightgbm as lgb
import numpy as np
from sklearn.metrics import accuracy_score, roc_auc_score
from wandb.lightgbm import wandb_callback
from optuna.integration.wandb import WeightsAndBiasesCallback


def get_accuracy(y, predictions):
    return accuracy_score(y, np.argmax(predictions, axis=1))


def get_auc(y, predictions):
    return roc_auc_score(lb.transform(y), predictions)


def objective(trial):
    X_train, X_validation, y_train, y_validation = train_test_split(X, y, test_size=0.25)
    train = lgb.Dataset(X_train, label=y_train)
    validation = lgb.Dataset(X_validation, label=y_validation)

    param = {
        "num_class": 3,
        "objective": "multiclass",
        "metric": "auc_mu",
        "verbosity": -1,
        "boosting_type": "gbdt",
        "lambda_l1": trial.suggest_float("lambda_l1", 1e-8, 10.0, log=True),
        "lambda_l2": trial.suggest_float("lambda_l2", 1e-8, 10.0, log=True),
        "num_leaves": trial.suggest_int("num_leaves", 2, 256),
        "feature_fraction": trial.suggest_float("feature_fraction", 0.4, 1.0),
        "bagging_fraction": trial.suggest_float("bagging_fraction", 0.4, 1.0),
        "bagging_freq": trial.suggest_int("bagging_freq", 1, 7),
    }

    pruning_callback = optuna.integration.LightGBMPruningCallback(trial, "auc_mu")
    gbm = lgb.train(
        param, train, valid_sets=[validation], callbacks=[pruning_callback, wandb_callback]
    )
    train_predictions = gbm.predict(X_train)
    validation_predictions = gbm.predict(X_validation)

    validation_auc = get_auc(y_validation, validation_predictions)
    param['train_auc'] = get_auc(y_train, train_predictions)
    param['validation_auc'] = validation_auc
    param['train_accuracy'] = get_accuracy(y_train, train_predictions)
    param['validation_accuracy'] = get_accuracy(y_validation, validation_predictions)
    param['Step'] = trial.number
    wandb.log(param)
    return validation_auc


wandbc = WeightsAndBiasesCallback(wandb_kwargs={"project": "example"})

study = optuna.create_study(study_name='experiment', direction='maximize')
study.optimize(objective, n_trials=10, callbacks=[wandbc])

print('Number of finished trials:', len(study.trials))
print('Best trial:', study.best_trial.params)

/Users/yonatanalexander/development/xdss/goldilox/.venv/lib/python3.7/site-packages/ipykernel_launcher.py:53: ExperimentalWarning: WeightsAndBiasesCallback is experimental (supported from v2.9.0). The interface can change in the future.


bagging_fraction,█▁
bagging_freq,▁▁
feature_fraction,█▁
lambda_l1,▁█
lambda_l2,█▁
num_class,▁▁
num_leaves,█▁
train_accuracy,█▁
train_auc,█▁
validation_accuracy,█▁
validation_auc,█▁


[I 2022-02-08 17:37:36,749] A new study created in memory with name: experiment
[I 2022-02-08 17:37:56,126] Trial 0 finished with value: 0.9798875581216802 and parameters: {'lambda_l1': 3.1835208743218324, 'lambda_l2': 0.0015129601392034257, 'num_leaves': 168, 'feature_fraction': 0.8015472800822189, 'bagging_fraction': 0.6965048482490156, 'bagging_freq': 5}. Best is trial 0 with value: 0.9798875581216802.
wandb: WARNING Step must only increase in log calls.  Step 0 < 1; dropping {'lambda_l1': 3.1835208743218324, 'lambda_l2': 0.0015129601392034257, 'num_leaves': 168, 'feature_fraction': 0.8015472800822189, 'bagging_fraction': 0.6965048482490156, 'bagging_freq': 5, 'value': 0.9798875581216802}.
[I 2022-02-08 17:38:08,822] Trial 1 finished with value: 0.9786793926366119 and parameters: {'lambda_l1': 1.6311852891444218, 'lambda_l2': 0.0036013398978833925, 'num_leaves': 154, 'feature_fraction': 0.7781225246926549, 'bagging_fraction': 0.7993277071494511, 'bagging_freq': 5}. Best is trial 0 w

Number of finished trials: 10
Best trial: {'lambda_l1': 0.0017430238577477218, 'lambda_l2': 0.010000712094551209, 'num_leaves': 76, 'feature_fraction': 0.9398065788388141, 'bagging_fraction': 0.5294344852069037, 'bagging_freq': 3}
